# GeoReady: On-Demand Spatial Data Prep for Any Canadian Community

### Import python modules

In [ ]:
import os
import requests
import zipfile
import ftplib
import geopandas as gpd
import ee
import geemap
import posixpath
import fiona
from tqdm import tqdm
from shapely.geometry import mapping

### Authenticate and Initialize Google Earth Engine and geemap

In [ ]:
## ee.Authenticate()
ee.Initialize(project='your-earth-engine-project-id-here')
geemap.ee_initialize()

### Define AOI

In [ ]:
subdivision_name = "Tumbler Ridge" 

### Set Paths

In [ ]:
# Set up directory paths
base_dir = os.path.dirname(os.getcwd())
raw_dir = os.path.join(base_dir, "data", "raw")
proc_dir = os.path.join(base_dir, "data", "processed")
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)

### PRUID-to-Folder Mapping

In [ ]:
# Matching the PRUID from Census Subdivions and matching to railway folders by province
pruid_to_folder = {
    '10': 'nl',   # Newfoundland and Labrador
    '11': 'pe',   # PEI (optional)
    '12': 'ns',   # Nova Scotia
    '13': 'nb',   # New Brunswick
    '24': 'qc',   # Quebec
    '35': 'on',   # Ontario
    '46': 'mb',   # Manitoba
    '47': 'sk',   # Saskatchewan
    '48': 'ab',   # Alberta
    '59': 'bc',   # British Columbia
    '60': 'yt',   # Yukon
    '61': 'nt',   # NWT
    '62': 'nu'    # Nunavut
}

### Download Canada Railroad Shapefiles from NRCan via Geo.ca

In [ ]:
# FTP connection details
ftp_host = "ftp.maps.canada.ca"
ftp_base_dir = "/pub/nrcan_rncan/vector/geobase_nrwn_rfn/"
download_dir = os.path.join(raw_dir, 'railroad_shapefiles')

# Ensure the download directory exists
os.makedirs(download_dir, exist_ok=True)

In [ ]:
# FTP Functions
def download_ftp_file(ftp, remote_file, local_file):
    with open(local_file, "wb") as f:
        ftp.retrbinary(f"RETR {remote_file}", f.write)


def list_ftp_dir(ftp, path):
    files = []
    ftp.retrlines(f"NLST {path}", files.append)
    return files


# FTP Functions
def download_shapefiles_from_ftp(ftp, base_dir, download_dir):
    # List the provinces in the base directory (e.g., 'ab', 'bc', etc.)
    provinces = list_ftp_dir(ftp, base_dir)

    for province in provinces:
        # Extract just the province code (e.g., 'yt' for Yukon)
        province_code = os.path.basename(province.strip('/'))
        province_dir = posixpath.join(base_dir, province_code)

        # List the files in the province directory
        files = list_ftp_dir(ftp, province_dir)

        # If no files are found for the province, skip it
        if not files:
            print(f"No files found for province {province_code}, skipping...")
            continue

        for file in files:
            # Only download English shapefiles (those ending with 'shp_en.zip')
            if file.endswith('shp_en.zip'):
                remote_file_path = posixpath.join(province_dir, file)

                # Correct the local path
                province_download_dir = os.path.join(download_dir, province_code)
                os.makedirs(province_download_dir, exist_ok=True)

                local_file_path = os.path.join(province_download_dir, os.path.basename(file))
                print(f"Downloading {remote_file_path}...")

                try:
                    download_ftp_file(ftp, remote_file_path, local_file_path)
                    print(f"Downloaded to {local_file_path}")

                    # Extract the zip file
                    with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
                        zip_ref.extractall(province_download_dir)
                        print(f"Extracted {local_file_path}")
                except Exception as e:
                    print(f"Failed to download or extract {file}: {e}")

In [ ]:
# Connect to FTP
ftp = ftplib.FTP(ftp_host)
ftp.login()

# Download shapefiles
download_shapefiles_from_ftp(ftp, ftp_base_dir, download_dir)

# Disconnect from FTP
ftp.quit()

### Download Industrial Sites from NRCan vie Geo.ca

In [ ]:
#Define the download directory inside your raw_dir 
download_dir = os.path.join(raw_dir)
os.makedirs(download_dir, exist_ok=True)

In [ ]:
# URL for the 900A dataset
url = "https://ftp.maps.canada.ca/pub/nrcan_rncan/Mining-industry_Industrie-miniere/900A_and_top_100/SHP/900A_74th_2024_shape.zip"

# Path to save the downloaded file
zip_path = os.path.join(download_dir, "900A_74th_shape.zip")

# Download and save the file
response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)

# Extract the ZIP file directly into the download directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all files without creating a subdirectory
    for file_name in zip_ref.namelist():
        zip_ref.extract(file_name, download_dir)

### Download Canada Census Subdivision Boundary zip file

In [ ]:
# URLs for downloading the Census Subdivision boundary zip file
csd_zip_url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcsd000a21a_e.zip"
csd_zip_path = os.path.join(raw_dir, "csd_2021_boundaries.zip")
csd_extract_dir = os.path.join(raw_dir, "csd_2021_boundaries")

In [ ]:
# Function to download zip files
def download_zip(url, zip_path):
    if not os.path.exists(zip_path):
        print(f"Downloading from {url}...")
        response = requests.get(url, verify=False)
        if response.status_code == 200:
            with open(zip_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {zip_path}!")
        else:
            raise Exception(f"Failed to download shapefile from {url}.")

# Download the CSD boundaries zip
download_zip(csd_zip_url, csd_zip_path)

### Extract CSD Boundary Shapefile

In [ ]:
# Extract the CSD boundary shapefile if not already extracted
if not os.path.exists(csd_extract_dir):
    with zipfile.ZipFile(csd_zip_path, 'r') as zip_ref:
        zip_ref.extractall(csd_extract_dir)
        print(f"Unzipped {csd_zip_path}!")


### Load Shapefile & Select AOI

In [ ]:
shp_file = [f for f in os.listdir(csd_extract_dir) if f.endswith(".shp")][0]
csd_gdf = gpd.read_file(os.path.join(csd_extract_dir, shp_file))
csd_match = csd_gdf[csd_gdf["CSDNAME"].str.lower() == subdivision_name.lower()]

if csd_match.empty:
    raise ValueError(f"No subdivision found with name '{subdivision_name}'")
else:
    print(f"Subdivision '{subdivision_name}' found!")


### Reproject CSD to WGS 84 for Earth Engine

In [ ]:
# Reproject to EPSG:4326 for Earth Engine
if csd_match.crs.to_epsg() != 4326:
    csd_match = csd_match.to_crs(epsg=4326)
    print("Reprojected to EPSG:4326 for GEE compatibility.")

### Convert to ee.Geometry

In [ ]:
geom_json = mapping(csd_match.geometry.values[0])
csd_ee_geom = ee.Geometry(geom_json)

### Match and Clip Railway Data to AOI

In [ ]:
# Determine PRUID and load matching railway shapefile
province_code = str(csd_match["PRUID"].iloc[0])
province_folder = pruid_to_folder.get(province_code)

if not province_folder:
    raise ValueError(f"Could not determine province folder for PRUID {province_code}")

# Build full filename based on province code
track_filename = f"NRWN_{province_folder}_2_0_TRACK.shp"
rail_path = os.path.join(raw_dir, "railroad_shapefiles", province_folder, track_filename)

if not os.path.exists(rail_path):
    raise FileNotFoundError(f"Railway track file not found at: {rail_path}")

# Load the shapefile
rail_gdf = gpd.read_file(rail_path)

# Ensure CRS matches
if rail_gdf.crs != csd_match.crs:
    rail_gdf = rail_gdf.to_crs(csd_match.crs)

# Clip railway data to CSD boundary
rail_clipped = gpd.clip(rail_gdf, csd_match)


### Match and Clip Industrial Sites to AOI

In [ ]:
# Load the industrial sites shapefiles and clip them to the AOI
industrial_sites = {
    "MetalWorks": "900A_74th_2024_MetalWorks.shp",
    "OilAndGas": "900A_74th_2024_OilAndGas.shp",
    "ProducingMines": "900A_74th_2024_ProducingMines.shp"
}

# Load the AOI (Census Subdivision geometry)
# Iterate through industrial sites and clip them to the AOI
clipped_industrial_sites = {}

for site_name, site_shapefile in industrial_sites.items():
    site_path = os.path.join(download_dir, "900A_74th_shape", site_shapefile)
    
    if os.path.exists(site_path):
        # Load the industrial site shapefile
        site_gdf = gpd.read_file(site_path)
        
        # Ensure CRS matches the AOI's CRS
        if site_gdf.crs != csd_match.crs:
            print(f"Reprojecting {site_name} from {site_gdf.crs} to {csd_match.crs}")
            site_gdf = site_gdf.to_crs(csd_match.crs)
        else:
            print("No crs match")
        
        # Clip the industrial site to the AOI
        clipped_site = gpd.clip(site_gdf, csd_match)
        
        # Store the clipped data in the dictionary
        clipped_industrial_sites[site_name] = clipped_site
        print(f"Clipped {site_name} to AOI.")
    else:
        print(f"File for {site_name} not found at {site_path}")

### Match and Clip Streams Data to AOI

In [ ]:
# Define paths, variables  and view metadata
hydrology_gpkg_path = os.path.join(raw_dir, "rhn_nhn_hhyd", "rhn_nhn_hhyd.gpkg")

available_layers = fiona.listlayers(hydrology_gpkg_path)
print("Available Hydrology Layers:", available_layers)

stream_layer = "nhn_hhyd_S_L_Watercourse_1"

with fiona.open(hydrology_gpkg_path, layer=stream_layer) as src:
    print("\nStream Layer Info:")
    print(f" - CRS: {src.crs}")
    print(f" - Geometry Type: {src.schema['geometry']}")
    print(f" - Properties: {list(src.schema['properties'].keys())}")
    print(f" - Number of features: {len(src)}")

In [ ]:
# Define paths and CSD boundary
clipped_stream_path = os.path.join(proc_dir, "clipped_streams.shp")
csd_geom = csd_match.union_all()

# Get bounding box of your AOI
bbox = csd_geom.bounds  # (minx, miny, maxx, maxy)
print(f"Bounding box of AOI: (minx: {bbox[0]:.2f}, miny: {bbox[1]:.2f}, maxx: {bbox[2]:.2f}, maxy: {bbox[3]:.2f})")


# Load stream layer using bounding box filter
print("Loading stream layer with bounding box filter...")
streams_gdf = gpd.read_file(
    hydrology_gpkg_path,
    layer=stream_layer,
    bbox=bbox  # Uses Fiona's internal filtering
)

print(f"Loaded {len(streams_gdf)} features within bounding box")

# Initialize progress bar for filtering by geometry intersection
print("Filtering features that intersect the CSD geometry...")
intersecting_streams = []

for _, row in tqdm(streams_gdf.iterrows(), total=len(streams_gdf), desc="Filtering", unit="feat", dynamic_ncols=True):
    if row.geometry.intersects(csd_geom):
        intersecting_streams.append(row)

# Create new GeoDataFrame from intersecting features
intersecting_streams_gdf = gpd.GeoDataFrame(intersecting_streams, crs=streams_gdf.crs)

# Save the result to a shapefile
intersecting_streams_gdf.to_file(clipped_stream_path)

# Final output
print(f"\nDone! {len(intersecting_streams_gdf)} stream segments that intersect the AOI saved to: {clipped_stream_path}")


### Fetch & Clip DEM from GEE

In [ ]:
# Get DEM data
dem_source = "USGS/SRTMGL1_003"
dem = ee.Image(dem_source)
clipped_dem = dem.clip(csd_ee_geom)

### Determine UTM Zone

In [ ]:
# Temporarily reproject to a projected CRS (World Mercator) for accurate centroid calculation
projected_csd = csd_match.to_crs(epsg=3395)  # From WGS 84 to World Mercator
centroid = projected_csd.geometry.centroid.iloc[0]

# Get longitude and latitude from the centroid (in geographic CRS)
# Transform the centroid back to lat/lon in WGS 84 for UTM zone calculation
centroid_lonlat = gpd.GeoSeries([centroid], crs=3395).to_crs(epsg=4326).geometry.iloc[0]
longitude = centroid_lonlat.x
latitude = centroid_lonlat.y

# Determine UTM zone based on the longitude
utm_zone = int((longitude + 180) // 6) + 1
hemisphere_code = 326 if latitude >= 0 else 327  # Northern or Southern Hemisphere
utm_crs = f"EPSG:{hemisphere_code}{utm_zone:02d}"

print(f"Determined UTM CRS: {utm_crs} (Zone {utm_zone}, {'Northern' if hemisphere_code == 326 else 'Southern'} Hemisphere)")

### Reproject All Layers to UTM Zone for Local Analysis Prep

In [ ]:
csd_match_utm = csd_match.to_crs(utm_crs)

clipped_dem_utm = clipped_dem.reproject(crs=utm_crs, scale=30)

# Create a new dictionary for reprojected industrial sites
clipped_industrial_sites_utm = {}
for site_name, site_gdf in clipped_industrial_sites.items():
    site_gdf_reprojected = site_gdf.to_crs(csd_match_utm.crs)
    # Update the new dictionary with the reprojected GeoDataFrame
    clipped_industrial_sites_utm[site_name] = site_gdf_reprojected
    
rail_clipped_utm = rail_clipped.to_crs(csd_match_utm.crs)

streams_clipped_utm = intersecting_streams_gdf.to_crs(csd_match_utm.crs)


print(f"CSD Boundary CRS: {csd_match_utm.crs}")
print(f"DEM CRS: {clipped_dem_utm.projection().getInfo()}")
for site_name, gdf in clipped_industrial_sites_utm.items():
    print(f"{site_name} CRS: {gdf.crs}")
print(f"Railways CRS: {rail_clipped_utm.crs}")
print(f"Streams CRS: {streams_clipped_utm.crs}")

### Export Processed Layers for QGIS Analysis

In [ ]:
# Define the output folder
export_dir = os.path.join("..", "data", "utm")
os.makedirs(export_dir, exist_ok=True)  # Make sure it exists

# Export CSD Boundary layer
csd_match_utm.to_file(os.path.join(export_dir, "csd_match_utm.shp"))

# Export DEM from ee
geemap.ee_export_image(
    clipped_dem_utm,
    os.path.join(export_dir, "clipped_dem_utm.tif"),
    scale=30,
    region=None, 
    file_per_band=False,
    crs="EPSG:32610"
)

# Export industrial sites layer
for name, gdf in clipped_industrial_sites_utm.items():
    out_path = os.path.join(export_dir, f"{name}_utm.shp")
    gdf.to_file(out_path)

# Export rail layer
rail_clipped_utm.to_file(os.path.join(export_dir, "rail_clipped_utm.shp"))

# Export stream layer
streams_clipped_utm.to_file(os.path.join(export_dir, "streams_clipped_utm.shp"))

### Map Display & Visualization Parameters

In [ ]:
# Set up the map centered on the subdivision geometry
Map = geemap.Map()
Map.centerObject(csd_ee_geom, zoom=9)

# Add CSD Boundary (AOI)
Map.addLayer(csd_ee_geom, {}, f"CSD Boundary")

# ---- DEM Visualization ----
# Get dynamic min/max elevation for DEM within the AOI
dem_min_utm = clipped_dem_utm.reduceRegion(
    reducer=ee.Reducer.min(), geometry=csd_ee_geom, maxPixels=1e8
).get("elevation").getInfo()

dem_max_utm = clipped_dem_utm.reduceRegion(
    reducer=ee.Reducer.max(), geometry=csd_ee_geom, maxPixels=1e8
).get("elevation").getInfo()

# Round the min/max values to the nearest whole number
dem_min_utm = round(dem_min_utm)
dem_max_utm = round(dem_max_utm)

# Set visualization parameters dynamically based on the min/max elevation
dem_vis_params = {
    'min': dem_min_utm,
    'max': dem_max_utm,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red'] 
}

# Add DEM Layer with dynamic parameters
Map.addLayer(clipped_dem_utm, dem_vis_params, f"DEM")

# Vector Layer Styling
rail_style = {
    'color': 'black',
    'weight': 4,    
    'opacity': 0.8,  
    'dashArray': '10,5'
}

streams_style = {
    'color': '#0066cc',   
    'weight': 1,       
    'opacity': 0.8,    
}

industrial_sites_color = "#808080"

Map.add_gdf(rail_clipped_utm, layer_name="Railways", info_mode=None, style=rail_style, zoom_to_layer=False)
Map.add_gdf(streams_clipped_utm, layer_name="Streams", info_mode=None, style=streams_style, zoom_to_layer=False)

for site_type, gdf in clipped_industrial_sites_utm.items():
    Map.add_gdf(
        gdf,
        layer_name=f"Ind. Sites: {site_type}",
        info_mode=None,
        style={"color": industrial_sites_color, "fillOpacity": 0.7, "weight": 2},
        zoom_to_layer=False  # Set to True if you want the map to zoom to the layer's extent
    )


# Add Legends
elevation_legend_dict = {
    f'Low Elevation ({dem_min_utm}m - {dem_min_utm + round((dem_max_utm - dem_min_utm)/5)}m)': '#0000FF',   # Blue
    f'Medium Low Elevation ({dem_min_utm + round((dem_max_utm - dem_min_utm)/5)}m - {dem_min_utm + round(2*(dem_max_utm - dem_min_utm)/5)}m)': '#00FF00',  # Green
    f'Medium High Elevation ({dem_min_utm + round(2*(dem_max_utm - dem_min_utm)/5)}m - {dem_min_utm + round(3*(dem_max_utm - dem_min_utm)/5)}m)': '#FFFF00',  # Yellow
    f'High Elevation ({dem_min_utm + round(3*(dem_max_utm - dem_min_utm)/5)}m - {dem_min_utm + round(4*(dem_max_utm - dem_min_utm)/5)}m)': '#FFA500',  # Orange
    f'Very High Elevation ({dem_min_utm + round(4*(dem_max_utm - dem_min_utm)/5)}m - {dem_max_utm}m)': '#FF0000',  # Red
}

vector_legend_dictionary = {
    "Industrial Sites (Points)": "#4A90E2",
    "Rail Tracks (Dashed Lines)": "#000000", 
"Streams (Lines)": '#0066cc'
}

Map.add_legend(title="Elevation", legend_dict=elevation_legend_dict)
Map.add_legend(title="Vector Layers", legend_dict=vector_legend_dictionary)

# Study area title
subdivision_name = csd_match["CSDNAME"].iloc[0]
Map.add_legend(title=f"Study Area: {subdivision_name}", legend_dict={}, position='bottomleft')

# Display the map
Map

### Hillshade

In [ ]:
hillshade = ee.Terrain.hillshade(clipped_dem_utm)

Map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade')

Map

### Slope

In [ ]:
slope = ee.Terrain.slope(clipped_dem_utm)

slope_stats = slope.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=csd_ee_geom,  # Correct geometry
    scale=30,
    bestEffort=True
)

# Print the output to inspect the dictionary structure
print(slope_stats.getInfo())

# Access the correct keys ('slope_min' and 'slope_max')
slope_min = slope_stats.get('slope_min').getInfo() 
max_slope = slope_stats.get('slope_max').getInfo()

# Define visualization parameters
slope_vis = {
    'min': slope_min,
    'max': max_slope * .9,
    'palette': ['white', 'yellow', 'orange', 'red']
}

slope_legend_dict = {
    'Low': '#FFFFFF',
    'Moderate': '#FFFF00',
    'Steep': '#FFA500',
    'Very Steep': '#FF0000'
}

# Add layers to the map
Map.addLayer(slope, slope_vis, 'Slope')
Map.add_legend(
    title='Slope',
    legend_dict=slope_legend_dict,
    position='bottomright'
)

# Display the map
Map.centerObject(clipped_dem_utm, 9)
Map

### Aspect

In [ ]:
aspect = ee.Terrain.aspect(clipped_dem_utm)

# Define function to reclassify aspect into directions
def classify_aspect(aspect_img):
    # Each range gets a direction ID (1 = N, 2 = NE, ..., 8 = NW)
    return (aspect_img
        .expression(
            """
            b('aspect') <= 22.5 || b('aspect') > 337.5 ? 1 :
            b('aspect') > 22.5 && b('aspect') <= 67.5 ? 2 :
            b('aspect') > 67.5 && b('aspect') <= 112.5 ? 3 :
            b('aspect') > 112.5 && b('aspect') <= 157.5 ? 4 :
            b('aspect') > 157.5 && b('aspect') <= 202.5 ? 5 :
            b('aspect') > 202.5 && b('aspect') <= 247.5 ? 6 :
            b('aspect') > 247.5 && b('aspect') <= 292.5 ? 7 :
            b('aspect') > 292.5 && b('aspect') <= 337.5 ? 8 : 0
            """)
        .rename('aspect_class')
    )

# Reclassify aspect
aspect_classified = classify_aspect(aspect).clip(csd_ee_geom)

# Define visualization parameters (custom palette for 8 directions)
aspect_palette = [
    'red',      # 1 = N
    'orange',   # 2 = NE
    'yellow',   # 3 = E
    'green',    # 4 = SE
    'blue',     # 5 = S
    'indigo',   # 6 = SW
    'violet',   # 7 = W
    'pink'      # 8 = NW
]

aspect_vis = {
    'min': 1,
    'max': 8,
    'palette': aspect_palette
}

# Define hex color codes for each aspect direction
aspect_legend_dict = {
    'North': '#FF0000',
    'Northeast': '#FFA500',
    'East': '#FFFF00',
    'Southeast': '#008000', 
    'South': '#0000FF',
    'Southwest': '#4B0082',
    'West': '#8A2BE2',   
    'Northwest': '#FFC0CB'
}

# Add to the map
Map.addLayer(aspect_classified, aspect_vis, 'Aspect')
Map.add_legend(
    title='Aspect Directions',
    legend_dict=aspect_legend_dict,
    position='bottomright'
)

# Display map
Map.centerObject(clipped_dem_utm, 9)
Map

### Final Raster Projection and Export

In [ ]:
# Hillshade
geemap.ee_export_image(
    hillshade,
    os.path.join(export_dir, "hillshade.tif"),
    scale=30,
    region = clipped_dem_utm.geometry(), 
    file_per_band=False,
    crs="EPSG:32610"
)

# Slope
geemap.ee_export_image(
    slope,
    os.path.join(export_dir, "slope.tif"),
    scale=30,
    region = clipped_dem_utm.geometry(), 
    file_per_band=False,
    crs="EPSG:32610"
)

# Aspect
geemap.ee_export_image(
    aspect,
    os.path.join(export_dir, "aspect.tif"),
    scale=30,
    region = clipped_dem_utm.geometry(), 
    file_per_band=False,
    crs="EPSG:32610"
)